In [1]:
!pip install tf-keras

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.3 MB/s eta 0:00:00


In [2]:
!pip install deepface

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------- ----------------- 0.8/1.4 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 5.0 MB/s eta 0:00:01
   -------------------------------------- - 1.8/1.9 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/332.0 MB 4.8 MB/s eta 0:01:10
   ---------------------------------------- 1.6/332.0 MB 4.2 MB/s eta 0:01:20
   ---------------------------------------- 2.4/332.0 MB 4.0 MB/s eta 0:01:23
   ---------------------------------------- 3.1/332.0 MB 3.9 MB/s eta 0:01:24
   ---------------------------------------- 3.9/332.0 MB 3.8 MB/s eta 0:01:26
    -----------------

In [3]:
!pip install pyzbar
!pip install "qrcode[pil]"

   ---------------------------------------- 0.0/817.4 kB ? eta -:--:--
   ---------------------------------------- 817.4/817.4 kB 5.7 MB/s eta 0:00:00


In [1]:
# import qrcode
# import cv2 as cv
# from pyzbar.pyzbar import decode


# def generate_qr_code(data, filename="qr_code.png", box_size=10, border=4):

#     qr = qrcode.QRCode(
#         version=1, 
#         error_correction=qrcode.constants.ERROR_CORRECT_H, 
#         box_size=box_size,
#         border=border,
#     )
    
#     # Add data to QR code
#     qr.add_data(data)
#     qr.make(fit=True)
    
#     # Create image
#     img = qr.make_image(fill_color="black", back_color="white")
    
#     # Save image
#     img.save(filename)
#     print(f"QR Code saved as {filename}")

# # Example usage:
# name = input("Enter student name: ")
# student_id = input("Enter student ID: ")
# face_image_path = input("Enter path to face image (or URL): ")

# # Create data dictionary
# student_data = {
#     "name": name,
#     "id": student_id,
#     "face_image": face_image_path
# }

# # Generate QR
# filename = f"{student_id}_qr.png"
# generate_qr_code(student_data, filename)


KeyboardInterrupt



In [6]:

#generate QR
import pandas as pd
import qrcode
import os
import smtplib
from email.message import EmailMessage

##SMTP Config
sender_email = "jhlee-wm22@student.tarc.edu.my"
sender_pass = "moseepjnzqxsdttn"
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()
server.login(sender_email, sender_pass)

# Load Excel
df = pd.read_excel("studentdb.xlsx")
output_dir = "qr_codes"
os.makedirs(output_dir, exist_ok=True)
for _, row in df.iterrows():
    student_id = row['student_id']
    name = row['name']
    email = row['email']

    # Data to store in QR
    if pd.isna(email) or "@" not in str(email):
        print(f"Warning----------- \n Skipping {name} (ID: {student_id}) — invalid or missing email")
        continue
    qr_data = f"{student_id}|{name}"

    # Create QR
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=10,
        border=4
    )
    qr.add_data(qr_data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")

    # Save QR at local
    qr_path = os.path.join(output_dir, f"{student_id}.png")
    img.save(os.path.join(output_dir, f"{student_id}.png"))

    #Email body
    msg = EmailMessage()
    msg["Subject"] = f"Your Student QR Code ({student_id})"
    msg["From"] = sender_email
    msg["To"] = email
    msg.set_content(
        f"Hello {name},\n\nAttached is your student QR code for graduation ceremony.\n\nBest regards, \n\n TARUMT"
    )

    with open(qr_path, "rb") as f:
        file_data = f.read()
        file_name = f"{student_id}.png"
        msg.add_attachment(file_data, maintype="image", subtype="png", filename=file_name)

    # Send email
    server.send_message(msg)
    print(f"Sent QR to {name} ({email})")

server.quit()
print("QR codes generated for all students.")

Sent QR to Lee Jia Hui (ljiahui010@gmail.com)
Sent QR to Kok Nai Yen (kokny-wm22@student.tarc.edu.my)
Sent QR to Heng Shi Yee (hengsy-wm22@student.tarc.edu.my)
Sent QR to nan (yeongcx-wm22@student.tarc.edu.my)
QR codes generated for all students.


In [8]:
import cv2 as cv
import pandas as pd
import face_recognition
from ultralytics import YOLO
from pyzbar.pyzbar import decode
import pyttsx3   # ✅ TTS library

# ------------------------
# Text-to-Speech Function
# ------------------------
def announce_name(student_name):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)   # adjust speaking speed
    engine.setProperty('volume', 1.0) # max volume
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)  # [0] male, [1] female (depends on system)
    
    text = f"Now presenting, {student_name}"
    print(f"🔊 Announcing: {student_name}")
    engine.say(text)
    engine.runAndWait()


# ------------------------
# QR Code Scan Function
# ------------------------
def scan_qr_and_get_student():
    df = pd.read_excel("studentdb.xlsx")

    cap = cv.VideoCapture(0)
    print("📷 Scanning for QR code... Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        decoded_objs = decode(frame)
        for obj in decoded_objs:
            qr_data = obj.data.decode('utf-8')
            student_id, name = qr_data.split('|')

            print(f"QR Detected: ID={student_id}, Name={name}")

            match = df[(df['student_id'] == student_id) & (df['name'] == name)]
            if not match.empty:
                image_path = match.iloc[0]['face_image_path']
                print(f"✅ Match found in Excel. Face image path: {image_path}")
                cap.release()
                cv.destroyAllWindows()
                return student_id, name, image_path
            else:
                print("❌ No match found in Excel.")

        cv.imshow("QR Scanner", frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv.destroyAllWindows()
            return None, None, None

    cap.release()
    cv.destroyAllWindows()
    return None, None, None


# ------------------------
# Face Recognition Function
# ------------------------
def face_match_with_qr(student_id, name, reference_image_path):
    model = YOLO("yolov8n.pt")

    # Load reference encoding
    ref_image = face_recognition.load_image_file(reference_image_path)
    ref_encoding = face_recognition.face_encodings(ref_image)[0]

    cap = cv.VideoCapture(0)
    print("🎥 Face recognition started... Press 'q' to quit.")

    matched = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, conf=0.5)

        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                face_roi = frame[y1:y2, x1:x2]

                try:
                    rgb_face = cv.cvtColor(face_roi, cv.COLOR_BGR2RGB)
                    enc = face_recognition.face_encodings(rgb_face)

                    if len(enc) > 0:
                        face_distances = face_recognition.face_distance([ref_encoding], enc[0])
                        conf = (1 - face_distances[0]) * 100 
                    
                        if conf >= 70 and not matched:   # ✅ only announce once
                            text = f"{name} ({student_id})  {conf:.2f}%"
                            color = (0, 255, 0)

                            matched = True
                            print(f"✅ Face match! ID={student_id}, Name={name}, Confidence={conf:.2f}%")
                           
                            cv.putText(frame, f"Name: {name}", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                            cv.putText(frame, f"ID: {student_id}", (50, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                            cv.imshow("Face Recognition", frame)
                            cv.waitKey(500) 

                            # 🔊 Announce the name here
                            announce_name(name)

                            # ✅ Instead of closing, just return so main loop goes back to QR scan
                            cap.release()
                            cv.destroyAllWindows()
                            return True 

                        else:
                            text = f"❌ Unmatched ({conf:.2f}%)"
                            color = (0, 0, 255)
                            print(f"❌ Face does not match QR code! Confidence={conf:.2f}%")
                    
                        # Draw bounding box + label
                        cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cv.putText(frame, text, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

                except Exception as e:
                    print("Error:", e)

        cv.imshow("Face Recognition", frame)

        if matched:  
            # ✅ Removed extra close here
            return True  

        if cv.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv.destroyAllWindows()
    return False


# ------------------------
# MAIN LOOP
# ------------------------
while True:
    student_id, name, image_path = scan_qr_and_get_student()
    if student_id is None:  
        break  
    
    result = face_match_with_qr(student_id, name, image_path)
    if result:
        print("Returning to QR scan for next student...")  # ✅ will auto refresh, not close
        continue  


📷 Scanning for QR code... Press 'q' to quit.
QR Detected: ID=240WMR09184, Name=Heng Shi Yee
✅ Match found in Excel. Face image path: C:\Users\User\Downloads\IP\IP\student_image\shiyee.jpg
🎥 Face recognition started... Press 'q' to quit.

0: 480x640 1 person, 64.8ms
Speed: 3.6ms preprocess, 64.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
❌ Face does not match QR code! Confidence=64.35%

0: 480x640 2 persons, 105.9ms
Speed: 2.4ms preprocess, 105.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
✅ Face match! ID=240WMR09184, Name=Heng Shi Yee, Confidence=75.68%
🔊 Announcing: Heng Shi Yee
Returning to QR scan for next student...
📷 Scanning for QR code... Press 'q' to quit.


In [3]:
!pip install pyttsx3

In [ ]:
# import dlib
# print(dlib.__version__)

# import face_recognition
# print(face_recognition.__version__)

# import cv2
# print(cv2.__version__)

20.0.0
1.2.3
4.12.0


In [3]:
# %pip install dlib face_recognition

  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Created wheel for dlib: filename=dlib-20.0.0-cp313-cp313-win_amd64.whl size=2964197 sha256=2b4c7d4b5b84e16bf724ac01ab70a75b42d2ddbeacc6bf0bb3e7bb67cdec48f4
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\d4\0d\ef\354663d702c37ab4b234126cfc6b61cfb36eda5ba193613a2c
Successfully built dlib

   ---------------------------------------- 0/3 [face-recognition-models]
   --------------------------------

In [5]:
# %pip install ultralytics

  Using cached ultralytics-8.3.185-py3-none-any.whl.metadata (37 kB)
  Using cached torchvision-0.23.0-cp313-cp313-win_amd64.whl.metadata (6.1 kB)
  Using cached ultralytics_thop-2.0.16-py3-none-any.whl.metadata (14 kB)
Using cached ultralytics-8.3.185-py3-none-any.whl (1.1 MB)
Using cached torchvision-0.23.0-cp313-cp313-win_amd64.whl (1.6 MB)
Using cached ultralytics_thop-2.0.16-py3-none-any.whl (28 kB)

   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   -------------------------- ------------- 2/3 [ultralytics]
   -------------------------- ---